<a href="https://colab.research.google.com/github/Pratyushk2003/resize-photo-from-landform-to-potrait/blob/main/RotateResize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from copy import deepcopy

In [54]:
import json

json_file_path = '/content/polotno - 2024-01-12T125913.339.json'

with open(json_file_path, 'r') as file:
    creative_json = json.load(file)
print(creative_json)

{'width': 1080, 'height': 1080, 'fonts': [], 'pages': [{'id': 'WIw0uPslYy', 'children': [{'id': 'DHR1fOEx7F', 'type': 'svg', 'name': '', 'opacity': 1, 'visible': True, 'selectable': True, 'removable': True, 'alwaysOnTop': False, 'showInExport': True, 'x': -32.09845131860976, 'y': 3.1833964978659067e-15, 'width': 1144.1969026372205, 'height': 535.7090977591724, 'rotation': 0, 'animations': [], 'blurEnabled': False, 'blurRadius': 10, 'brightnessEnabled': False, 'brightness': 0, 'sepiaEnabled': False, 'grayscaleEnabled': False, 'shadowEnabled': False, 'shadowBlur': 5, 'shadowOffsetX': 0, 'shadowOffsetY': 0, 'shadowColor': 'black', 'shadowOpacity': 1, 'draggable': True, 'resizable': True, 'contentEditable': True, 'styleEditable': True, 'src': 'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCA2MCA2MCI+PHJlY3QgeD0iMCIgeT0iMCIgd2lkdGg9IjYwIiBoZWlnaHQ9IjYwIiBmaWxsPSJyZ2IoMCwgMTYxLCAyNTUpIiAvPjwvc3ZnPg==', 'maskSrc': '', 'cropX': 0, 'cropY': 0, 'c

In [55]:
import numpy as np

In [56]:
def get_new_dim(rot, w, h):
    rad = np.deg2rad(rot)
    new_w = np.abs(w * np.cos(rad)) + np.abs(h * np.sin(rad))
    new_h = np.abs(h * np.cos(rad)) + np.abs(w * np.sin(rad))
    return new_w, new_h

def get_outer_bbox(layer):
    orig_coord = get_final_coordinates(layer)
    new_dim = get_new_dim(layer["rotation"], layer["width"], layer["height"])
    x_o, y_o = orig_coord
    x_c, y_c = x_o + layer["width"]/2, y_o + layer["height"]/2
    x_outer, y_outer = x_c - new_dim[0]/2, y_c - new_dim[1]/2
    return x_outer, y_outer, new_dim[0], new_dim[1]

def find_diag_len(w, h):
    diag = np.sqrt(w ** 2 + h ** 2)
    return diag


def get_given_coords_in_standard_sys(rotation, radius, offset_angle):
    rot_rad = np.deg2rad(rotation + offset_angle)
    new_x = (radius * np.sin(rot_rad))
    new_y = (radius * np.cos(rot_rad))

    return new_x, new_y


def calculate_delta(x, y, ref_x, ref_y):
    delta_x = x - ref_x
    delta_y = y - (ref_y * -1.0)  # invert the system

    return delta_x, delta_y


def get_final_coordinates(layer, need_extra=False):
    w = layer["width"]
    h = layer["height"]
    rotation_deg = layer["rotation"]
    x = layer["x"]
    y = layer["y"]
    radius = find_diag_len(w, h) / 2
    offset_angle = -1.0 * np.rad2deg(np.arcsin((w / 2) / radius))
    ref_x, ref_y = get_given_coords_in_standard_sys(rotation_deg, radius, offset_angle)
    delta_x, delta_y = calculate_delta(x, y, ref_x, ref_y)
    new_x, new_y = get_given_coords_in_standard_sys(offset_angle, radius, 0)
    final_x = new_x + delta_x
    final_y = -1.0 * new_y + delta_y  # invert the co-ordinate system
    if need_extra:
        return final_x, final_y, {"offset_angle": offset_angle, "radius": radius}
    return int(np.ceil(final_x)), int(np.ceil(final_y))

In [57]:
#maintain aspect ratio
def magic_resize_template_json(creative_json, new_h, new_w):
    old_w = creative_json.get("width")
    old_h = creative_json.get("height")
    old_aspect = old_w / old_h
    old_area = old_w * old_h

    new_template = deepcopy(creative_json)
    new_template["width"] = new_w
    new_template["height"] = new_h
    new_aspect = new_w / new_h
    new_area = new_w * new_h
    pages = new_template.get("pages", [])
    for page_index in range(0, len(pages)):
        childrens = new_template.get("pages")[page_index].get("children")
        for child in childrens:
            if child["type"] == "svg":

                    '''if child_area <= 0.2 * old_area:
                        child["old_x"] = child["x"]
                        child["old_y"] = child["y"]
                        child["old_width"] = child["width"]
                        child["old_height"] = child["height"]
                        child["x"] = (new_w/2) - (child["width"]/2) - (((old_w/2) - (child["old_x"] + (child["old_width"]/2)))*new_w/old_w)
                        child["y"] = (new_h/2) - (child["height"]/2) - (((old_h/2) - (child["old_y"] + (child["old_height"]/2)))*new_h/old_h)
                    '''

                    child["old_x"] = child["x"]
                    child["old_y"] = child["y"]
                    child["old_width"] = child["width"]
                    child["old_height"] = child["height"]
                    child["width"] = child["width"] * new_w / old_w
                    child["height"] = child["height"] * new_h / old_h
                    child["x"] = (new_w/2) - (child["width"]/2) - (((old_w/2) - (child["old_x"] + (child["old_width"]/2)))*new_w/old_w)
                    child["y"] = (new_h/2) - (child["height"]/2) - (((old_h/2) - (child["old_y"] + (child["old_height"]/2)))*new_h/old_h)
                    #rotate the image back to original angle

            elif child["type"] == "image":
                if child["name"] == "logo":
                  if child["rotation"] != 0:
                      child["x"],child["y"],child["width"],child["height"]=get_outer_bbox(child)
                  else:
                    left = 0
                    right = old_w
                    top = 0
                    bottom = old_h
                    child["old_x"] = child["x"]
                    child["old_y"] = child["y"]
                    child["old_width"] = child["width"]
                    child["old_height"] = child["height"]
                    if (abs(child["old_x"])+child["width"]) < right-2 and (abs(child["old_y"]) + child["height"]) < bottom-2 and child["old_y"] > top+2 and child["old_x"] > left+2:
                        child["width1"] = child["width"] * new_w / old_w
                        child["height1"] = child["width1"] * child["height"]/child["width"]
                        child["height2"] = child["height"] * new_h / old_h
                        child["width2"] = child["height2"] * child["width"]/child["height"]
                        a1logo = child["width1"]*child["height1"]
                        a2logo = child["width2"]*child["height2"]
                        if a1logo < a2logo:
                          child["width"] = child["width1"]
                          child["height"] = child["height1"]

                        else:
                          child["width"] = child["width2"]
                          child["height"] = child["height2"]

                    elif abs(child["old_x"])+child["width"] >= right-2 and (abs(child["old_y"]) + child["height"]) >= bottom-2 and child["old_y"] <= top+2 and child["old_x"] <= left+2:
                      child["height"] = new_h + abs(child["y"]) + child["height"] - bottom
                      child["width"] = new_w + abs(child["x"]) + child["width"] - right

                    else:
                      child["width"] = child["width"] * new_w / old_w
                      child["height"] = child["height"] * new_h / old_h
                    child["x"] = (new_w/2) - (child["width"]/2) - (((old_w/2) - (child["old_x"] + (child["old_width"]/2)))*new_w/old_w)
                    child["y"] = (new_h/2) - (child["height"]/2) - (((old_h/2) - (child["old_y"] + (child["old_height"]/2)))*new_h/old_h)
                else:
                      rad = np.deg2rad(child['rotation'])
                      b = np.abs(child['width']*np.cos(rad))
                      a = np.abs(child['height'] * np.sin(rad))
                      f = np.abs(child['height']*np.cos(rad))
                      g = np.abs(child['width'] * np.sin(rad))

                      child['x'],child['y'],child['width'],child['height'] =  get_outer_bbox(child)
                      print("child after rotation",child['x'],child['y'],child['width'],child['height'],child['rotation'])
                      child["old_x"] = child["x"]
                      child["old_y"] = child["y"]
                      child["old_width"] = child["width"]
                      child["old_height"] = child["height"]
                      child["width1"] = child["width"] * new_w / old_w
                      child["height1"] = child["width1"] * child["height"]/child["width"]
                      child["height2"] = child["height"] * new_h / old_h
                      child["width2"] = child["height2"] * child["width"]/child["height"]
                      a1svg = child["width1"]*child["height1"]
                      a2svg = child["width2"]*child["height2"]

                      if a1svg < a2svg:
                        child["width"] = child["width1"]
                        child["height"] = child["height1"]
                        #print("a1svg :",a1svg)
                      else:
                        child["width"] = child["width2"]
                        child["height"] = child["height2"]
                      child["x"] = (new_w/2) - (child["width"]/2) - (((old_w/2) - (child["old_x"] + (child["old_width"]/2)))*new_w/old_w)
                      child["y"] = (new_h/2) - (child["height"]/2) - (((old_h/2) - (child["old_y"] + (child["old_height"]/2)))*new_h/old_h)


                      if child['rotation']>0 and child['rotation']<90:
                        c = a/b
                        d = child['width']*c/(1+c)
                        child['x'] = child['x'] + d
                        child['height'] = np.abs(d /np.sin(rad))
                        child['width'] = np.abs((child['width'] - d)/np.cos(rad))

                      elif child['rotation'] == 90:
                        e  = child['width']
                        child["x"] = child["x"] + child['width']
                        child['width'] = child['height']
                        child['height'] = e

                      elif child['rotation']>90 and child['rotation']<180:
                        rad = 180 - rad
                        child["x"] = child["x"] + child['width']
                        h = f/g
                        i = child['height']*h/(1+h)
                        child["y"] = child["y"] + i
                        child['height'] = np.abs(i /np.cos(rad))
                        child['width'] = np.abs((child['height'] - i)/np.sin(rad))

















            elif child["type"] == "text":
                child["old_x"] = child["x"]
                child["old_y"] = child["y"]
                child["old_width"] = child["width"]
                child["old_height"] = child["height"]
                child["width"] = child["width"] * new_w / old_w
                child["height"] = child["height"] * new_h / old_h
                child["x"] = (new_w/2) - (child["width"]/2) - (((old_w/2) - (child["old_x"] + (child["old_width"]/2)))*new_w/old_w)
                child["y"] = (new_h/2) - (child["height"]/2) - (((old_h/2) - (child["old_y"] + (child["old_height"]/2)))*new_h/old_h)

                max_fsz = int(min(150, child.get("fontSize", 30) * 1.5))
                try:
                    new_f, text_h = find_correct_fontsize_v3(child["width"], child["height"],
                                                             child["text"],
                                                             child["fontFamily"],
                                                             child.get("lineHeight", 1.0),
                                                             child.get("letterSpacing", 0),
                                                             max_font_sz=max(max_fsz, child["fontSize"]))
                    child["fontSize"] = new_f
                except Exception:
                    pass
            else:
                child["old_x"] = child["x"]
                child["old_y"] = child["y"]
                child["old_width"] = child["width"]
                child["old_height"] = child["height"]
                child["width"] = child["width"] * new_w / old_w
                child["height"] = child["height"] * new_h / old_h
                child["x"] = (new_w/2) - (child["width"]/2) - (((old_w/2) - (child["old_x"] + (child["old_width"]/2)))*new_w/old_w)
                child["y"] = (new_h/2) - (child["height"]/2) - (((old_h/2) - (child["old_y"] + (child["old_height"]/2)))*new_h/old_h)
    return new_template

In [58]:
new_template = magic_resize_template_json(creative_json, 1080, 1920)

child after rotation 198.00000000000006 296.0 675.0050498877171 643.9065488951021 0
child after rotation 198.14875051443556 296.5100658804048 675.0050498877184 643.9065488951021 140.0386540629902


In [59]:
print(new_template)

{'width': 1920, 'height': 1080, 'fonts': [], 'pages': [{'id': 'WIw0uPslYy', 'children': [{'id': 'DHR1fOEx7F', 'type': 'svg', 'name': '', 'opacity': 1, 'visible': True, 'selectable': True, 'removable': True, 'alwaysOnTop': False, 'showInExport': True, 'x': -57.06391345530629, 'y': 0.0, 'width': 2034.1278269106142, 'height': 535.7090977591724, 'rotation': 0, 'animations': [], 'blurEnabled': False, 'blurRadius': 10, 'brightnessEnabled': False, 'brightness': 0, 'sepiaEnabled': False, 'grayscaleEnabled': False, 'shadowEnabled': False, 'shadowBlur': 5, 'shadowOffsetX': 0, 'shadowOffsetY': 0, 'shadowColor': 'black', 'shadowOpacity': 1, 'draggable': True, 'resizable': True, 'contentEditable': True, 'styleEditable': True, 'src': 'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCA2MCA2MCI+PHJlY3QgeD0iMCIgeT0iMCIgd2lkdGg9IjYwIiBoZWlnaHQ9IjYwIiBmaWxsPSJyZ2IoMCwgMTYxLCAyNTUpIiAvPjwvc3ZnPg==', 'maskSrc': '', 'cropX': 0, 'cropY': 0, 'cropWidth': 1, 'crop

In [60]:
def save_to_json_file(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
save_to_json_file(new_template, '/content/polotno.json')

In [61]:
from google.colab import files
files.download('/content/polotno.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>